In [ ]:
from lxml import objectify
import pandas as pd
import matplotlib.pyplot as plt
import itertools
%matplotlib inline

In [ ]:
def plot(x,y=None):
    if y==None:
        tmp = range(len(x))
        y = x
        x = tmp
    fig, ax = plt.subplots(figsize=(16,2))
    ax.plot(x,y)
    plt.show()

In [ ]:
path=r'dataset/REFIT_BUILDING_SURVEY.xml'
tree = objectify.parse(path)
root = tree.getroot()
NS={'a':'http://www.refitsmarthomes.org'}

path=r'dataset/REFIT_TIME_SERIES_VALUES.csv'

try:
    csv
except:
    csv=pd.read_csv(path, index_col=0, parse_dates=[1])

In [ ]:
#Energy consumption in home
def get_energy():
    """
        Available only for 1st building, due to needs a lot of memory on drive
    """
    
    path = r'dataset/CLEAN_House1.csv'
    csvnrg=pd.read_csv(path, header=0,usecols=["Time", "Aggregate"], parse_dates=[0])

    data_en = csvnrg['Aggregate'].tolist()
    time_en = csvnrg['Time'].tolist()

#     plot(time_en, data_en)
    
    return data_en, time_en

In [ ]:
#Values of (gas based) Radiator temperature sensor
def get_radiator(building_num=1, rad_num=2):
    appliance = 'Radiator[%d]' % (rad_num)
    NS={'a':'http://www.refitsmarthomes.org'}
    
    radiators = root.xpath('./a:Stock/a:Building[%d]/a:%s' % (building_num, appliance), namespaces=NS)

    data_rad = []
    time_rad = []

    for rad in radiators:
        r_id = rad.get('id')
        t = []
        v = []
        for sensor in rad.getchildren():
            for time_series_variable in sensor.getchildren():
                id = time_series_variable.get('id')
                t.append(csv.loc[id]['dateTime'].tolist())
                v.append(csv.loc[id]['data'].values.tolist())
        data_rad = list(itertools.chain.from_iterable(v))
        time_rad = list(itertools.chain.from_iterable(t))
    
    return data_rad, time_rad

In [ ]:
#Summs values for intervals

by_day = lambda t: t.day
by_week = lambda t: t.week
by_month = lambda t: t.month
    
act_sum = lambda s, vi: s + vi
act_max = lambda s, vi: max(s,vi)
act_min = lambda s, vi: min(s,vi)
    
def act_interval(check_param, action, times, values):
    t = []
    v = []
    s = values[0]
    for i in range(1, len(times)):
        if check_param(times[i]) == check_param(times[i-1]):
            s = action(s, values[i])
        else:
            v.append(s)
            t.append(times[i])
            s = values[i]
    return t,v

In [ ]:
#Usage of gas in home

def get_gas(building_num=1):
    appliance = 1 #Number of gas meter
    NS={'a':'http://www.refitsmarthomes.org'}
    tsv = root.xpath('./a:Stock/a:Building[%d]/a:Meter[%d]/a:Sensor/a:TimeSeriesVariable' % (building_num, appliance), namespaces=NS)

    id = tsv[0].get('id')

    time_gas = csv.loc[id]['dateTime'].tolist()
    data_gas = csv.loc[id]['data'].values.tolist()
    
    time_gas, data_gas = act_interval(by_day, act_sum, time_gas, data_gas)
    
    return data_gas, time_gas


def get_fft(data):
    from numpy.fft import rfft
    fg = rfft(data).real
#     print('gas day sum FFT coefficients')
#     plot(fg)
#     len(fg)
    
    return fg

In [ ]:
#Weather data 
def get_clim():
    NS={'a':'http://www.refitsmarthomes.org'}
    elements=root.xpath('./a:Stock/a:Climate/a:Sensor/a:TimeSeriesVariable', namespaces=NS)
    data_clim = []
    time_clim = []
    for e in elements:
        id = e.get('id')
        variable_type=e.get('variableType')
        units=e.get('units')
        t,v = act_interval(by_day, act_min, csv.loc[id]['dateTime'].tolist(), csv.loc[id]['data'].tolist())
        data_clim.append(v)
        if len(time_clim) == 0:
            time_clim = t
        else: 
            assert len(time_clim) == len(t)
    
    return data_clim, time_clim
#         ax.plot(csv.loc[id]['dateTime'],csv.loc[id]['data'])
#         plt.show()

In [ ]:
# e = elements[0]
# climate_sensors = []
# for i in elements:
#     climate_sensors.append(i.attrib['variableType'])
# print(climate_sensors)
# #indexes: 0, 1, 2, -2, -1

# Data preprocessing

In [ ]:
#TODO: try LSTM model to predict gas consumption 

#Changeable parameters:
# building_num = 2      #Begins from 1 (not 0) ):
# gap = 7               #len (days before now) for FFT (for input layer of LSTM)

def prepare_data(building_num=2, gap=7):
    data_gas, time_gas = get_gas(building_num)
    data_clim, time_clim = get_clim()

#     print('Initial data:')
#     print('len(gas): ', len(time_gas))
#     print('len(clim): ', len(time_clim))

    #align dates in intervals
    #meter must be @gap days before now
    #climate must begin day after now (tommorow)

#     print('First possible date of gas (with gap shifting): ', time_gas[gap])
#     print('First date of clim: ', time_clim[0])
    begin = time_gas.index(time_clim[0]) - gap
    time_gas = time_gas[begin:]
    data_gas = data_gas[begin:]

    end = time_clim.index(time_gas[-1]) + 1
    time_clim = time_clim[:end]

#     print('After formatting: ')
#     print(len(time_gas))
#     print(len(time_clim))
#     print('first')
#     print(time_gas[gap])
#     print(time_clim[0])

#     print('last')
#     print(time_gas[-1])
#     print(time_clim[-1])

    tmp = []
    for d in data_clim:
        tmp.append(d[:end])
    data_clim = tmp
    
    #Prepare data to train LSTM

    xinp = [] #Construct of 0-3: FFT coefficients, 4-weather forecast values on tommorow
    ypred = []
    
    from numpy.fft import rfft
    for idx in range(gap, len(time_gas)-1):
        item = time_gas[idx]
        row = rfft(data_gas[idx-gap:idx]).real.tolist()
        ypred.append(data_gas[idx+1])
        #add weather forecast
        widx = time_clim.index(item) #weather's series index for current date
        row.append(data_clim[0][widx+1])
        row.append(data_clim[1][widx+1])
        row.append(data_clim[2][widx+1])
        row.append(data_clim[-2][widx+1])
        row.append(data_clim[-1][widx+1])

        xinp.append(row)

#     print(xinp[-1])
#     print(ypred[-1])
    
    return xinp, ypred

In [ ]:
#Store input and output
import pickle
import os.path

def get_data(building_num=2, gap=7):
    finp = 'inp_building=%d_gap=%d.pkl' % (building_num, gap)
    fout = 'out_building=%d_gap=%d.pkl' % (building_num, gap)

    if os.path.exists(finp) and os.path.exists(fout):
        with open('inp.pkl', 'rb') as f:
            X = pickle.load(f)
        with open('pred.pkl', 'rb') as f:
            Y = pickle.load(f)
    else:
        X,Y = prepare_data(building_num, gap)
        
        #Store
        with open(finp, 'wb') as f:
            pickle.dump(X, f)
        with open(fout, 'wb') as f:
            pickle.dump(Y, f)
            
    return numpy.array(X).astype('float32'), numpy.array(Y).astype('float32').reshape(-1,1)

        
        
        
def saveAsText(xinp, ypred):
    with open('inp.txt', 'w') as f:
        for i in xinp:
            f.write(', '.join(map(str, i)))
            f.write('\n')
    with open('out.txt', 'w') as f:
        for i in ypred:
            f.write(str(i))
            f.write('\n')

# RNN (LSTM) building

In [ ]:
#Train LSTM

import numpy
import matplotlib.pyplot as plt
import pandas
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras import backend as K
import random
random.seed(42)

In [ ]:
# frameX = pandas.read_csv('inp.txt', engine='python')
# setX = frameX.values
# setX = setX.astype('float32')

building   = 2
gap        = 5
split      = 0.67
neurons    = 100
epochs     = 30
batch_size = 10

setX, setY = get_data(building, gap)

# normalize the setX
scalerX = MinMaxScaler(feature_range=(0, 1))
setX = scalerX.fit_transform(setX)

# split into train and test sets
train_size = int(len(setX) * split)
test_size = len(setX) - train_size
train, test = setX[0:train_size,:], setX[train_size:len(setX),:]

# convert an array of values into a setX matrix
trainX = numpy.reshape(train, (train.shape[0], 1, train.shape[1]))
testX = numpy.reshape(test, (test.shape[0], 1, test.shape[1]))

#Scale

scalerY = MinMaxScaler(feature_range=(0, 1))
setY = scalerY.fit_transform(setY)
trainY, testY = setY[0:train_size,:], setY[train_size:len(setY),:]
    

In [ ]:
#design and fit our LSTM network
   
model = Sequential()
model.add(LSTM(neurons, input_shape=(trainX.shape[1], trainX.shape[2])))
model.add(Dropout(0.2))
# model.add(LSTM(neurons))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

history = model.fit(trainX, trainY, epochs=epochs, batch_size=batch_size, validation_data=(testX, testY), verbose=0, shuffle=False)

# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')
plt.show()
    
    
# make predictions
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)
# invert predictions
trainPredict = scalerY.inverse_transform(trainPredict)
trainInvY = scalerY.inverse_transform(trainY)
testPredict = scalerY.inverse_transform(testPredict)
testInvY = scalerY.inverse_transform(testY)
# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(trainInvY, trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testInvY, testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

Successfull param combinations: 
 * gap        = 5
 * split      = 0.67
 * neurons    = 100
 * epochs     = 40
 * batch_size = 10
 * Train Score: 0.94 RMSE
 * Test Score: 1.10 RMSE


In [ ]:
# make predictions
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)
# invert predictions
trainPredict = scalerY.inverse_transform(trainPredict)
trainInvY = scalerY.inverse_transform(trainY)
testPredict = scalerY.inverse_transform(testPredict)
testInvY = scalerY.inverse_transform(testY)
# calculate root mean squared error
tmp = mean_squared_error(trainInvY, trainPredict[:,0])
trainScore = math.sqrt(tmp)
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testInvY, testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

In [ ]:
# shift train predictions for plotting
trainPredictPlot = numpy.empty_like(setY)
trainPredictPlot[:, :] = numpy.nan
trainPredictPlot[:len(trainPredict), :] = trainPredict
# shift test predictions for plotting
testPredictPlot = numpy.empty_like(setY)
testPredictPlot[:, :] = numpy.nan
testPredictPlot[len(trainPredict):len(setY), :] = testPredict
# plot baseline and predictions
fig, ax = plt.subplots(figsize=(18,6))
ax.plot(scalerY.inverse_transform(setY))
ax.plot(trainPredictPlot)
ax.plot(testPredictPlot)
plt.show()

In [ ]:
a = list(map(int, scalerY.inverse_transform(setY[len(trainPredict):])))
b = list(map(int, testPredict))
fig, ax = plt.subplots(figsize=(18,6))
ax.plot(a)
ax.plot(b)
plt.show()

In [ ]:
## to release gpu

def free_gpu():
    try:
        del model
    except:
        print('No model')
    try:
        del history
    except:
        print('No history')
    import gc
    K.clear_session()
    for i in range(15): gc.collect()

free_gpu()